In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
# import just the Exeter data
exeter_data = pd.read_csv("Exeter_city_only.csv", header = 0)

In [18]:
exeter_data["CP"].unique()

array([ 6407, 16023, 16382, 16971, 16972, 18081, 26410, 37674, 37882,
       47077, 56192, 56414, 77996, 77997, 99845, 99846])

Ideally for each of these points we will have 3 sets of coordinates which we can then plots some flows for. 

In [8]:
exeter_data[["AADFYear","Road","CP","StartJunction","EndJunction","LinkLength_km"]][0:17]

,AADFYear,Road,CP,StartJunction,EndJunction,LinkLength_km
0,2000,A379,6407,Sannerville Way Rndbt,Glasshouse Lane,0.9
1,2000,M5,16023,30,29,1.8
2,2000,A379,16382,Bridge road/Sannerville Way Rndbt,A38 Mid Junction,3.5
3,2000,A377,16971,A3015 Western Way,B3183 New North Road,2.1
4,2000,A377,16972,B3123,A3015 Nr Frog St,1.7
5,2000,A377,18081,A30,/B3123,0.6
6,2000,A379,26410,A3015 Topsham Road,A3015,0.8
7,2000,A376,37674,M5 Jct 30,M5,0.7
8,2000,A30,37882,A377,M5 Jct 31,3.6
9,2000,A377,47077,B3183 New North Road,A396,1.2


In [56]:
# Want to make a dataframe for each road. Each road now has a set of points associated with it. 

# Loop over each of the CP numbers and make one dataframe out of them

Exeter_CPs = exeter_data["CP"].unique()
Exeter_roads = exeter_data["Road"][exeter_data["AADFYear"] == 2000]
Exeter_length = exeter_data["LinkLength_miles"][exeter_data["AADFYear"] == 2000]
Exeter_cat = exeter_data["RoadCategory"][exeter_data["AADFYear"] == 2000]

def make_df(CP):
    road_df = pd.read_csv("%s_all.csv" %str(CP), 
                              names = ["CP","GridReference","Easting","Northing","Latitude","Longitude"],
                              usecols = [0,1,2,3,4,5],header = 0)
    return road_df


In [57]:
store_dfs = []

for i, CP in enumerate(Exeter_CPs):
    df = make_df(CP)
    df["CP"] = CP
    # add the roads, Link Length and Road Category to the dataframe. Just using the values from the first year
    df["Road"] = Exeter_roads[i]
    df["LinkLength_miles"] = Exeter_length[i]
    df["RoadCategory"] = Exeter_cat[i]
    
    store_dfs.append(df)

# Now append
store_dfs = pd.concat(store_dfs, axis=0) 

In [58]:
store_dfs[0:14]

,CP,GridReference,Easting,Northing,Latitude,Longitude,Road,LinkLength_miles,RoadCategory
0,6407,SX 93715 89043,293715,89043,50.691292,-3.506076,A379,0.55,PR
1,6407,SX 93841 89119,293841,89119,50.691999,-3.504317,A379,0.55,PR
2,6407,SX 93955 89231,293955,89231,50.693032,-3.502729,A379,0.55,PR
3,6407,SX 94288 89633,294288,89633,50.696702,-3.498137,A379,0.55,PR
4,6407,SX 94180 89559,294180,89559,50.696023,-3.499639,A379,0.55,PR
5,6407,SX 94088 89498,294088,89498,50.695452,-3.500926,A379,0.55,PR
6,6407,SX 94026 89439,294026,89439,50.694908,-3.501785,A379,0.55,PR
7,6407,SX 94010 89384,294010,89384,50.694419,-3.501999,A379,0.55,PR
0,16023,SX 96870 93012,296870,93012,50.727546,-3.462539,M5,1.11,TM
1,16023,SX 96861 92245,296861,92245,50.720645,-3.462453,M5,1.11,TM


Now we have a string of points along each CP. So we have mapped some roads. 
Now we want to plot these on a map!

In [59]:
# Save the new dataframe of points for later use
store_dfs.to_csv('road_coords.csv', index = False)

In [50]:
road_coords = pd.read_csv("road_coords.csv", header = 0)

In [51]:
print (road_coords.count())

CP               213
GridReference    213
Easting          213
Northing         213
Latitude         213
Longitude        213
dtype: int64
